코사인 거리 기반의 kmeans clustering을 위한 코드

In [ ]:
!pip install sentence-transformers datasets

In [ ]:
!pip install umap-learn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#이 부분 자기 폴더 경로에 맞게 바꾸기
%cd drive/My\ Drive/Colab\ Notebooks/BERT/datas

In [ ]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sentence_transformers import models

In [ ]:
import scipy
import numpy  as np
import pickle
import matplotlib.pyplot as plt
import os

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

In [ ]:
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture

In [ ]:
import umap

In [ ]:
#문서 파일 다운로드
!gdown "1-F2Qgk83uyob9Fg2Od4F2ez4IW2Lb4tQ&confirm=t"

In [ ]:
#파일 저장 경로로 바꾸기
docs = np.load('/content/drive/MyDrive/Colab Notebooks/BERT/datas/newdata_6_4.npy', allow_pickle=True)
docs = pd.Series(docs)
#각자 20만개씩 쪼개서 돌리기
docs = docs[:200000]

In [ ]:
#클러스터 그래프가 엄청 이쁘게 나오는 이유는 이것들이 쿼리이기 때문.
from numpy.linalg import norm
%matplotlib inline

def cosine(x, y):
  return np.dot(x, y)/(norm(x) * norm(y))


def get_scores(X, labels):
  CHI = calinski_harabasz_score(X, labels)
  DBI = davies_bouldin_score(X, labels)
  return (CHI, DBI)

model_list = ["sentence-transformers/all-mpnet-base-v2", 'sentence-transformers/multi-qa-distilbert-cos-v1', "sentence-transformers/all-distilroberta-v1"]
#'PCA', 'SVD',
DR_list = ['UMAP']
# 'dbscan', , 'hdbscan', 'gmm'
clustering_list = ['kmeans']

for name in model_list:
  model_name = name
  embedding_model = models.Transformer(model_name)
  pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=True,
    pooling_mode_max_tokens=False,
  )
  model = SentenceTransformer(model_name)
  model_name_re = ' '.join(model_name.split('/'))

  #자기 폴더 경로에 맞게 바꾸기 -> 나중에 피클 파일 저장되면 이 코드는 빼버리세요
  document_embeddings = model.encode(docs)
  with open('/content/drive/MyDrive/Colab Notebooks/BERT/datas/6_4_tweets_embedded_by_'+ model_name_re + '.txt', 'wb') as f:
    pickle.dump(document_embeddings, f)

  #자기 폴더 경로에 맞게 바꾸기
  with open('/content/drive/MyDrive/Colab Notebooks/BERT/datas/6_4_tweets_embedded_by_'+ model_name_re + '.txt', 'rb') as f:
    document_embeddings = pickle.load(f)

  #자기 폴더 경로에 맞게 바꾸기
  directory = "/content/drive/MyDrive/Colab Notebooks/BERT/results/" +  model_name_re
  os.mkdir(directory)

  for DR_name in DR_list:
    dr_directory = directory + '/' +' DR: ' + DR_name
    os.mkdir(dr_directory)

    #PCA는 공분산을 이용해서 데이터의 분포와 단위가 다르면 스케일링 할 필요가 있지만 임베딩된 데이터를 살펴보니 그럴 필요가 없어보임
    if DR_name == 'PCA':
      X = PCA(n_components=2).fit_transform(document_embeddings)

    if DR_name == 'SVD':
      X = TruncatedSVD(n_components = 2).fit_transform(document_embeddings)

    if DR_name == 'UMAP':
      X = umap.UMAP().fit_transform(document_embeddings)

    for clustering_name in clustering_list:
      if DR_name == 'SVD' and clustering_name == 'kmeans':
        continue
      cluster_directory = dr_directory + '/' + 'cluster: ' + clustering_name
      os.mkdir(cluster_directory)

      text_folder = cluster_directory + '/' + 'sentences'
      image_folder = cluster_directory + '/' + 'images'
      os.mkdir(text_folder)
      os.mkdir(image_folder)

      if clustering_name == 'dbscan':

        #이 부분 반복문으로 eps하고 min_samples 변화시켜야 함
        dbscan = DBSCAN(eps=0.5, min_samples=2)
        dbscan.fit(X)
        cluster_assignment = dbscan.labels_
        cls_dist=pd.Series(cluster_assignment).value_counts()

        #이 부분 일단 클러스터 중심을 뽑을 수가 있어서 거리기반으로 재봤는데 코사인 기반으로도 해봐야 할 듯
        distances = scipy.spatial.distance.cdist(dbscan.core_sample_indices_, X)
        centers={}

        f = open(text_folder + '/' + str(num_clusters) + ' clusters' + '.txt', 'w')

        #각 지표별 score 구하기
        CHI, DBI = get_scores(X, dbscan.labels_)

        f.write(f'davies_bouldin_score: {DBI}, calinski_harabasz_score: {CHI}')
        f.write('\n')
        f.write('#####################################')
        f.write('\n')

        for j, d in enumerate(distances):
          f.write(str(j) + '번째 클러스터')
          f.write('-----------------------------------')
          f.write('\n')
          ind = np.argsort(d, axis=0)[:20]
          for i, idx in enumerate(ind):
            f.write(str(i) + '번째로 가까운 문장: ' + docs[idx])
            f.write('\n')
          f.write('\n')
        f.close()

        labels= dbscan.labels_
        fig, ax = plt.subplots(figsize=(12,12))
        plt.scatter(X[:,0], X[:,1], c=labels, s=1, cmap='Paired')
        plt.title('model: ' + model_name + ', clusters: ' + str(num_clusters) + ', DR: ' + DR_name + ', cluster: ' + clustering_name)
        plt.colorbar()
        plt.savefig(image_folder + '/' + 'number of cluster: ' + str(num_clusters) + '.png')

      else:
        for num_clusters in range(5, 16):
          #코사인 기반 kmeans
          if clustering_name == 'kmeans':
            length = np.sqrt((X**2).sum(axis=1))[:,None]
            X = X / length

            kmeans = KMeans(n_clusters=num_clusters).fit(X)

            len_ = np.sqrt(np.square(kmeans.cluster_centers_).sum(axis=1)[:,None])
            centers = kmeans.cluster_centers_ / len_
            dist = 1 - np.dot(centers, X.T) # K x N matrix of cosine distances

            cluster_assignment = kmeans.labels_
            cls_dist=pd.Series(kmeans.labels_).value_counts()

            clustered_sentences = [[] for _ in range(num_clusters)]

            for sentence_id, cluster_id in enumerate(cluster_assignment):
                clustered_sentences[cluster_id].append(docs[sentence_id])

            distances = scipy.spatial.distance.cdist(centers, X)

            f = open(text_folder + '/' + str(num_clusters) + ' clusters' + '.txt', 'w')

            #각 지표별 score 구하기
            CHI, DBI = get_scores(X, kmeans.labels_)

            f.write(f'davies_bouldin_score: {DBI}, calinski_harabasz_score: {CHI}')
            f.write('\n')
            f.write('#####################################')
            f.write('\n')

            for j, d in enumerate(distances):
              f.write(str(j) + '번째 클러스터')
              f.write('-----------------------------------')
              f.write('\n')

              ind = np.argsort(d, axis=0)
              center = ind[0]

              f.write('클러스터 중심 문장: ' + docs[center])
              f.write('\n')

              for i, idx in enumerate(ind[1:10]):
                f.write(str(i) + '번째로 가까운 문장: ' + docs[idx])
                f.write('\n')

              f.write('####################################')
              f.write('\n')

              for i, idx in enumerate(ind[-1:-11:-1]):
                f.write(str(i) + '번째로 먼 문장: ' + docs[idx])
                f.write('\n')

              f.write('####################################')
              f.write('\n')
            f.close()

            labels= kmeans.labels_
            fig, ax = plt.subplots(figsize=(12,12))
            plt.scatter(X[:,0], X[:,1], c=labels, s=1, cmap='Paired')
            plt.title('model: ' + model_name + ', clusters: ' + str(num_clusters) + ', DR: ' + DR_name + ', cluster: ' + clustering_name)
            plt.colorbar()
            plt.savefig(image_folder + '/' + 'number of cluster: ' + str(num_clusters) + '.png')

          elif clustering_name == 'hdbscan':
            hdbscan = hdbscan.HDBSCAN(min_cluster_size=num_clusters)
            cluster_assignment = hdbscan.fit_predict(X)

            cls_dist=pd.Series(cluster_assignment).value_counts()
            clusters = np.unique(cls_dist.index)

            clustered_sentences = [[] for _ in range(num_clusters)]

            for sentence_id, cluster_id in enumerate(cluster_assignment):
                clustered_sentences[cluster_id].append(docs[sentence_id])

             #각 지표별 score 구하기
            CHI, DBI = get_scores(X, hdbscan.labels_)

            f = open(text_folder + '/' + str(num_clusters) + ' clusters' + '.txt', 'w')

            f.write(f'davies_bouldin_score: {DBI}, calinski_harabasz_score: {CHI}')
            f.write('\n')
            f.write('#####################################')
            f.write('\n')

            for cluster in clusters:
              f.write(str(cluster) + '번째 클러스터')
              f.write('-----------------------------------')
              f.write('\n')

              for idx, sentence in enumerate(clustered_sentences[cluster][:20]):
                f.write(str(idx) + '번째 문장: ' + sentence)
                f.write('\n')
              f.write('\n')
            f.close()

            labels= hdbscan.labels_
            fig, ax = plt.subplots(figsize=(12,12))
            plt.scatter(X[:,0], X[:,1], c=labels, s=1, cmap='Paired')
            plt.title('model: ' + model_name + ', clusters: ' + str(num_clusters) + ', DR: ' + DR_name + ', cluster: ' + clustering_name)
            plt.colorbar()
            plt.savefig(image_folder + '/' + 'number of cluster: ' + str(num_clusters) + '.png')

          elif clustering_name == 'gmm':
            gmm = GaussianMixture(n_components=num_clusters)
            gmm.fit(X)
            cluster_assignment = gmm.predict(X)
            cls_dist=pd.Series(cluster_assignment).value_counts()
            clusters = np.unique(cls_dist.index)

            clustered_sentences = [[] for _ in range(num_clusters)]
            sentence_indices = [[] for _ in range(num_clusters)]

            number_of_sentences = 10

            for sentence_id, cluster_id in enumerate(cluster_assignment):
                clustered_sentences[cluster_id].append(docs[sentence_id])
                #일단 number_of_sentences개씩 밑에서 보여주므로 number_of_sentences개 문장을 저장한다.
                if len(sentence_indices[cluster_id]) < number_of_sentences:
                  sentence_indices[cluster_id].append(sentence_id)

            #GMM은 군집 별 중심이 없기 때문에 그냥 클러스터 내의 텍스트만 뽑아내자
            f = open(text_folder + '/' + str(num_clusters) + ' clusters' + '.txt', 'w')

            #각 지표별 score 구하기
            CHI, DBI = get_scores(X, cluster_assignment)

            f.write(f'davies_bouldin_score: {DBI}, calinski_harabasz_score: {CHI}')
            f.write('\n')
            f.write('#####################################')
            f.write('\n')

            for cluster in clusters:
              f.write(str(cluster) + '번째 클러스터')
              f.write('-----------------------------------')
              f.write('\n')

              for idx, sentence in enumerate(clustered_sentences[cluster][:number_of_sentences]):
                nth_sentence_idx = sentence_indices[cluster][idx]

                f.write(str(idx) + '번째 기준 문장: ' + sentence)
                f.write('\n')

                a = []
                target = document_embeddings[nth_sentence_idx]

                for idx, sentence in enumerate(document_embeddings):
                  a.append((cosine(target, sentence), idx))

                a.sort(key = lambda x: x[0], reverse = True)

                for idx in range(1, 6):
                  f.write(str(idx) + '번째로 유사도가 높은 문장' +  ': ' + docs[a[idx][1]])
                  f.write('\n')
                  f.write('코사인 유사도: ' + str(a[idx][0]))
                  f.write('\n')

                f.write('#####################################')
                f.write('\n')

                for idx in range(1, 6):
                  f.write(str(idx) + '번째로 유사도가 낮은 문장' +  ': ' + docs[a[-idx][1]])
                  f.write('\n')
                  f.write('코사인 유사도: ' + str(a[-idx][0]))
                  f.write('\n')

                f.write('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
                f.write('\n')

              f.write('\n')
            f.close()

            labels= cluster_assignment
            fig, ax = plt.subplots(figsize=(12,12))
            plt.scatter(X[:,0], X[:,1], c=labels, s=1, cmap='Paired')
            plt.title('model: ' + model_name + ', clusters: ' + str(num_clusters) + ', DR: ' + DR_name + ', cluster: ' + clustering_name)
            plt.colorbar()
            plt.savefig(image_folder + '/' + 'number of cluster: ' + str(num_clusters) + '.png')

